Assignment 3 : Real-time Color detection
Build accurate real-time multi-colors object detection
- Detect objects in real-time using a webcam.
- Detect objects by distinguishing them by color, at least 3 colors.
- Draw a frame around the object, specifying the color of the frame to match the color
of the object, and write a message naming the color.
- Can count the number of objects that appear in each color.

In [2]:
import cv2
import pandas as pd
import numpy as np

In [43]:

cap = cv2.VideoCapture(0)
cv2.namedWindow("Output")

def detect_color(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    colors = {
        "Orange": ([0, 200, 120], [10, 255, 255]),
        "Yellow": ([15, 100, 100], [30, 255, 255]),
        "Cyan": ([85, 100, 100], [140, 255, 255])
    }
    
    detected_objects = []
    color_counts = {"Orange": 0, "Yellow": 0, "Cyan": 0}
    
    for color, (lower, upper) in colors.items():
        lower = np.array(lower, dtype=np.uint8)
        upper = np.array(upper, dtype=np.uint8)
        mask = cv2.inRange(hsv, lower, upper)
        kernel = np.ones((5,5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        for cnt in contours:
            if cv2.contourArea(cnt) > 2000:  # Ignore small areas
                x, y, w, h = cv2.boundingRect(cnt)
                detected_objects.append((x, y, w, h, color))
                color_counts[color] += 1
    return detected_objects, color_counts

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    flipped_frame = cv2.flip(frame, 1)
    objects, color_counts = detect_color(flipped_frame)
    
    for (x, y, w, h, color) in objects:
        color_map = {"Orange": (0, 165, 255), "Yellow": (0, 255, 255), "Cyan": (255, 255, 0)}
        cv2.rectangle(flipped_frame, (x, y), (x + w, y + h), color_map[color], 2)
        cv2.putText(flipped_frame, color, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_map[color], 2)
    
    text_y = 20
    for color, count in color_counts.items():
        cv2.putText(flipped_frame, f"{color}: {count}", (10, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        text_y += 20
    
    cv2.imshow("Output", flipped_frame)
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

cap.release()
cv2.destroyAllWindows()
